<a href="https://colab.research.google.com/github/skevin-dev/NLP-FELLOWSHIP/blob/week5/Assignment_(1).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# First Model Assignment
The aim of this assignment is to make sure you understand the foundations of model training. We have covered traditional ML and also simple NN. You task is 
1. to write code for training traditional ML model that gives the highest accuracy
2. Code for NN model that gives the highest accuracy

## This we will consider
1. The code works
2. The hyperparameter used for fine tuning - epoch only is not enough
3. The highest accuracy
4. Bonus points for being creative with preprocessing, tokenization and creation of vectors

# Submissions
1. Notebook with code
2. 2 models

# Deadline
Monday 28th at 5pm

In [ ]:
!pip install clean-text


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 53 kB 959 kB/s 
     |████████████████████████████████| 175 kB 64.5 MB/s 
  Created wheel for emoji: filename=emoji-1.7.0-py3-none-any.whl size=171046 sha256=3b9143186a0ebffde7a9dfd47ff1fa5cd6de41e97c5f9e287ed0db40f9733b23
  Stored in directory: /root/.cache/pip/wheels/8a/4e/b6/57b01db010d17ef6ea9b40300af725ef3e210cb1acfb7ac8b6
Successfully built emoji


In [ ]:
!pip install torch==1.8.0 torchtext==0.9.0 #compatibility

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 735.5 MB 16 kB/s 
     |████████████████████████████████| 7.1 MB 55.7 MB/s 
  Attempting uninstall: torch
    Found existing installation: torch 1.12.1+cu113
    Uninstalling torch-1.12.1+cu113:
      Successfully uninstalled torch-1.12.1+cu113
  Attempting uninstall: torchtext
    Found existing installation: torchtext 0.13.1
    Uninstalling torchtext-0.13.1:
      Successfully uninstalled torchtext-0.13.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.13.1+cu113 requires torch==1.12.1, but you have torch 1.8.0 which is incompatible.
torchaudio 0.12.1+cu113 requires torch==1.12.1, but you have torch 1.8.0 which is incompatible.


## Libraries 

In [ ]:
import pandas as pd 
import numpy as np 
import os 
import re
import string 
from cleantext import clean
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
import joblib
from sklearn.model_selection import GridSearchCV


#models 
from sklearn.ensemble import RandomForestClassifier


#accuracy
from sklearn.metrics import accuracy_score,classification_report



In [ ]:
#mount google drive 
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#accessing data directoty 
os.chdir('/content/drive/MyDrive/NLP Fellowship /data')


In [ ]:
!ls

50k_imdb_movie_reviews.csv  Eng_Kin-Paralleldata.csv
article.txt		    saved_weights_linear.pt


In [ ]:
#import data 
df = pd.read_csv('50k_imdb_movie_reviews.csv')

In [ ]:
#shape of the dataset to be used 
df.shape 

(50000, 3)

In [ ]:
# checking and removing duplicates 
df.duplicated(subset =['review']).sum()

418

In [ ]:
# drop duplicates 
df.drop_duplicates(subset=['review'],inplace=True)

In [ ]:
#head of the dataset 
df.head()

,review,sentiment,set
0,I went and saw this movie last night after bei...,1,test
1,Actor turned director Bill Paxton follows up h...,1,test
2,As a recreational golfer with some knowledge o...,1,test
3,"I saw this film in a sneak preview, and it is ...",1,test
4,Bill Paxton has taken the true story of the 19...,1,test


In [ ]:
df['review'][4]

'Bill Paxton has taken the true story of the 1913 US golf open and made a film that is about much more than an extra-ordinary game of golf. The film also deals directly with the class tensions of the early twentieth century and touches upon the profound anti-Catholic prejudices of both the British and American establishments. But at heart the film is about that perennial favourite of triumph against the odds.<br /><br />The acting is exemplary throughout. Stephen Dillane is excellent as usual, but the revelation of the movie is Shia LaBoeuf who delivers a disciplined, dignified and highly sympathetic performance as a working class Franco-Irish kid fighting his way through the prejudices of the New England WASP establishment. For those who are only familiar with his slap-stick performances in "Even Stevens" this demonstration of his maturity is a delightful surprise. And Josh Flitter as the ten year old caddy threatens to steal every scene in which he appears.<br /><br />A old fashioned

we can see it contains some numbers 

## data cleaning 

In [ ]:
#using cleaning function 
def cleaning_text(text):
  """This functions helps to make some cleanings given a text file 

  Args
  ----
  text(str): text to be cleaned 

  Returns
  -------
  final_text(list): returns a list containing cleaned text  
  """

  # convert text file into a list and remove empty lines 
  uncleaned_text = [line for line in text.splitlines() if line.strip()]

  # same cases 
  same_case = [line.lower() for line in uncleaned_text]

  # remove duplicate lines 
  no_dups_line = []
  for line in same_case:
    if line not in no_dups_line:
      no_dups_line.append(line)

  # remove emails and urls
  email_link_pattern = r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b|http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'

  no_email_link = [re.sub(email_link_pattern,"",line) for line in no_dups_line ]

  # remove punctuations
  no_punct = [line.translate(str.maketrans(" "," ",string.punctuation)) for line in no_email_link]

  # remove emojis 
  no_emojis = [clean(line,no_emoji=True) for line in no_punct]

  # remove sensitive data 
  sensitive_data_pattern =  r'\d{1,}|\+\d+\s\d+\s\d+|\+\d+\s\d+'

  no_sensitive_data = [re.sub(sensitive_data_pattern,"",line) for line in no_emojis ]

  # remove white spaces 
  final_text = [" ".join(line.split()) for line in no_sensitive_data]

  final_text_text  = final_text[0]

  return final_text_text

In [ ]:
#apply data cleaning function on review column
df['review'] = df['review'].apply(cleaning_text)

In [ ]:
# check 
df['review'][4]

'bill paxton has taken the true story of the us golf open and made a film that is about much more than an extraordinary game of golf the film also deals directly with the class tensions of the early twentieth century and touches upon the profound anticatholic prejudices of both the british and american establishments but at heart the film is about that perennial favourite of triumph against the oddsbr br the acting is exemplary throughout stephen dillane is excellent as usual but the revelation of the movie is shia laboeuf who delivers a disciplined dignified and highly sympathetic performance as a working class francoirish kid fighting his way through the prejudices of the new england wasp establishment for those who are only familiar with his slapstick performances in even stevens this demonstration of his maturity is a delightful surprise and josh flitter as the ten year old caddy threatens to steal every scene in which he appearsbr br a old fashioned movie in the best sense of the 

we can see that it is cleaned 

In [ ]:
# spliting training dataset and testing dataset 
training_dataset =  df[df['set'] == 'train']
testing_dataset = df[df['set']=='test']
print(f'the shape of training dataset is {training_dataset.shape}')
print(f'the shape of testing dataset is {testing_dataset.shape}')


the shape of training dataset is (24781, 3)
the shape of testing dataset is (24801, 3)


In [ ]:
# delete set columns 
del training_dataset['set']
testing_dataset.drop('set',axis =1,inplace=True)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [ ]:
print(f'the shape of training dataset is {training_dataset.shape}')
print(f'the shape of testing dataset is {testing_dataset.shape}')

the shape of training dataset is (24781, 2)
the shape of testing dataset is (24801, 2)


In [ ]:
training_dataset.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 24781 entries, 25000 to 49999
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   review     24781 non-null  object
 1   sentiment  24781 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 580.8+ KB


In [ ]:
# spliting data into training and validation 
X,y = training_dataset['review'],training_dataset['sentiment']
X_train,X_val,y_train,y_val = train_test_split(X,y,test_size = 0.20,random_state = 42)

In [ ]:
# vectorization using TF-IDF 

vectorizer = TfidfVectorizer()
X_train_vec =  vectorizer.fit_transform(X_train)
joblib.dump(vectorizer,'/content/drive/MyDrive/NLP Fellowship /Models/TFIDFvector.pkl')
X_val_vec = vectorizer.transform(X_val)
X_test_vec = vectorizer.transform(testing_dataset['review'])

In [ ]:
X_train_vec.shape

(19824, 102865)

In [ ]:
# Vectorization using countvector
vc = CountVectorizer()
X_train_vc = vc.fit_transform(X_train)
joblib.dump(vc,'/content/drive/MyDrive/NLP Fellowship /Models/countvector.pkl')
X_val_vc = vc.transform(X_val)
X_test_vc = vc.transform(testing_dataset['review'])

## Tuning tradition ML model and their parameters using Scikit-learn 

In [ ]:
n_estimators = [i for i in range(10,110,10)]
criterion = ['gini','entropy']
max_depth = [i for i in range(1,11)]

In [ ]:
for n_estimator in n_estimators:
  for crit in criterion:
    for m in max_depth:
      #instantaite the model with number of estimators as parameter 
      model =  RandomForestClassifier(n_estimators = n_estimator,criterion=crit,max_depth=m)

      # train the model
      model.fit(X_train_vec,y_train)

      # predict using training and validation 
      train_pred = model.predict(X_train_vec)
      val_pred =  model.predict(X_val_vec)

      #accuracy 
      training_acc = sum(train_pred==y_train)*1.0/len(y_train)
      validation_acc =  sum(val_pred==y_val)*1.0/len(y_val)

      # results 
      print("parameters:({},{},{}); trainining accuracy:{}; validation accuracy:{}".format(n_estimator,crit,m,training_acc,validation_acc))


parameters:(10,gini,1); trainining accuracy:0.5658797417271993; validation accuracy:0.5755497276578576
parameters:(10,gini,2); trainining accuracy:0.6211158192090396; validation accuracy:0.6114585434738753
parameters:(10,gini,3); trainining accuracy:0.6262106537530266; validation accuracy:0.6211418196489812
parameters:(10,gini,4); trainining accuracy:0.7004640839386602; validation accuracy:0.6887230179544079
parameters:(10,gini,5); trainining accuracy:0.6949656981436643; validation accuracy:0.6865039338309461
parameters:(10,gini,6); trainining accuracy:0.7358757062146892; validation accuracy:0.724430098850111
parameters:(10,gini,7); trainining accuracy:0.7278551251008878; validation accuracy:0.6986080290498285
parameters:(10,gini,8); trainining accuracy:0.7306295399515739; validation accuracy:0.6943715957232197
parameters:(10,gini,9); trainining accuracy:0.7487389023405973; validation accuracy:0.7177728464797256
parameters:(10,gini,10); trainining accuracy:0.768815577078289; validation

gini performs better than entropy in terms of criterion, the model performs well as we increase n_estimators and max_depth 

In [ ]:
# let's pass other parameters and test the model on testing dataset 
model_ = RandomForestClassifier(n_estimators = 500, criterion ='gini',max_depth = 50)

model_.fit(X_train_vec,y_train)

train_prediction =  model_.predict(X_train_vec)

test_prediction =  model_.predict(X_test_vec)

training_acc = sum(train_prediction==y_train)*1.0/len(y_train)
testing_acc =  sum(test_prediction==testing_dataset['sentiment'])*1.0/len(testing_dataset['sentiment'])

print('training_accuracy: {}  testing_accuracy: {}'.format(training_acc,testing_acc))

training_accuracy: 0.9961158192090396  testing_accuracy: 0.8452884964315955


In [ ]:
training_dataset.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 24781 entries, 25000 to 49999
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   review     24781 non-null  object
 1   sentiment  24781 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 580.8+ KB


In [ ]:
# model serialization 
joblib.dump(model_,'/content/drive/MyDrive/NLP Fellowship /Models/RandomForest.pkl')

['/content/drive/MyDrive/NLP Fellowship /Models/RandomForest.pkl']

## Tuning Linear regression and MLP using pytorch 

### Linear Regression(LR)

In [ ]:
import torch
from torchtext.legacy import data
from torchtext.legacy.data import Dataset, Example
from torchtext.legacy.data import BucketIterator
import torch.nn as nn
import torch.nn.functional as F

In [ ]:
SEED = 42

torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True # Check this
max_document_length = 300 #hyperparameter

TEXT = data.Field(lower=True, include_lengths=True,  tokenize='spacy',batch_first=True,  fix_length=max_document_length)
LABEL = data.Field(sequential=False, use_vocab=False)

class DataFrameDataset(Dataset):
    def __init__(self, df: pd.DataFrame, fields: list):
        super(DataFrameDataset, self).__init__(
            [
                Example.fromlist(list(r), fields) 
                for i, r in df.iterrows()
            ], 
            fields
        )

/usr/local/lib/python3.7/dist-packages/torchtext/data/utils.py:123: UserWarning: Spacy model "en" could not be loaded, trying "en_core_web_sm" instead
  warnings.warn(f'Spacy model "{language}" could not be loaded, trying "{OLD_MODEL_SHORTCUTS[language]}" instead')


In [ ]:
torch_valid_dataset, torch_test_dataset = DataFrameDataset(
    df=testing_dataset, 
    fields=(
        ('review', TEXT),
        ('sentiment', LABEL)
    )
).split() 

In [ ]:
torch_train_dataset = DataFrameDataset(
    df=training_dataset, 
    fields=(
        ('review', TEXT),
        ('sentiment', LABEL)
    )
)

In [ ]:
max_size = 30000 #hyperparameter
TEXT.build_vocab(torch_train_dataset, max_size=max_size,vectors='fasttext.simple.300d')
vocab_size = len(TEXT.vocab)

BATCH_SIZE = 40 #hyperparameter
train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (torch_train_dataset, torch_valid_dataset, torch_test_dataset), 
    batch_size = BATCH_SIZE ,
    sort_key=lambda x: len(x.review),
    sort_within_batch=True)

In [ ]:
class LR(nn.Module):
    def __init__(self, input_size, hidden_size,hidden_size2,hidden_size3, num_classes):
        super(LR, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size) # 
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, hidden_size2)
        self.fc3 = nn.Linear(hidden_size2, hidden_size3) 
        self.fc4 = nn.Linear(hidden_size3, num_classes)


    def forward(self, text):
        text = text.float() # dense layer deals just with float type data
        x = self.fc1(text) #(m x n) with (n x p)
        x = self.relu(x)
        x = self.relu(self.fc2(x))
        x = self.relu(self.fc3(x))
        
        preds = self.fc4(x) # crossentropyloss handles the softmax
        # preds = F.softmax(preds,1) # nn.softmax
        return preds

In [ ]:
class MLP(nn.Module):
    def __init__(self, vocab_size, embed_size, hidden_size2, hidden_size3, hidden_size4, output_dim, dropout, max_document_length):
        super().__init__()
        # embedding and convolution layers
        self.embedding = nn.Embedding(vocab_size, embed_size)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(dropout)
        self.fc1 = nn.Linear(embed_size*max_document_length, hidden_size2)  # dense layer
        self.fc2 = nn.Linear(hidden_size2, hidden_size3)  # dense layer
        self.fc3 = nn.Linear(hidden_size3, hidden_size4)  # dense layer
        self.fc4 = nn.Linear(hidden_size4, output_dim)  # dense layer

    def forward(self, text):
         # text shape = (batch_size, num_sequences)
        embedded = self.embedding(text)
        # embedded = [batch size, sent_len, emb dim]
        
        x = embedded.view(embedded.shape[0], -1)  # x = Flatten()(x)
        #embedded = embedded.unsqueeze(1) # fc gets 4 dimension
        
        x = self.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.relu(self.fc2(x))
        x = self.dropout(x)
        x = self.relu(self.fc3(x))
        x = self.dropout(x)
        preds = self.fc4(x)
        # preds = F.softmax(preds, 1)
        return preds

In [ ]:
LRlr = 1e-3
LRbatch_size = 100
LRdropout_keep_prob = 0.3
LRembedding_size = 300
LRmax_document_length = 300 # each sentence has until 100 words
LRvocab_size = len(TEXT.vocab)
LRdev_size = 0.8 # split percentage to train\validation data
LRmax_size = 30000 # maximum vocabulary size
LRseed = 30
LRnum_classes = 2

LRnum_epochs = 20
LRhidden_size = 300
LRhidden_size1 = 400
LRhidden_size2 = 200
LRhidden_size3 = 80

to_train = True
LinearRmodel = LR(LRmax_document_length, LRhidden_size,LRhidden_size2,LRhidden_size3, LRnum_classes)

In [ ]:
lr = 1e-3
batch_size = 100
dropout_keep_prob = 0.3
embedding_size = 300
max_document_length = 300 # each sentence has until 100 words
vocab_size = len(TEXT.vocab)
dev_size = 0.8 # split percentage to train\validation data
max_size = 30000 # maximum vocabulary size
seed = 30
num_classes = 2

num_epochs = 15
hidden_size = 256
hidden_size1 = 300
hidden_size2 = 128
hidden_size3 = 64

to_train = True
MLP_model = MLP(vocab_size, embedding_size, hidden_size1, hidden_size2, hidden_size3,  num_classes, dropout_keep_prob, max_document_length)

In [ ]:
def accuracy(probs, target):
  winners = probs.argmax(dim=1)
  corrects = (winners == target)
  accuracy = corrects.sum().float() / float(target.size(0))
  return accuracy

In [39]:
best_valid_loss = float('inf')
optimizer = torch.optim.Adam(MLP_model.parameters(), lr=lr)

loss_func = nn.CrossEntropyLoss()

for epoch in range(LRnum_epochs):
  train_epoch_loss = 0
  train_epoch_acc = 0
  for batch in train_iterator:
      optimizer.zero_grad()
      # retrieve text and no. of words
      text, text_lengths = batch.review

      #feedforward
      # model.to(device)
      predictions = MLP_model(text).squeeze(1)
      
      
      loss = loss_func(predictions, batch.sentiment)

      acc = accuracy(predictions, batch.sentiment)

      # perform backpropagation
      loss.backward()

      optimizer.step()

      train_epoch_loss += loss.item()
      train_epoch_acc += acc.item()

  

  valid_epoch_loss = 0
  valid_epoch_acc = 0

  MLP_model.eval()

  with torch.no_grad():
      for batch in valid_iterator:
          text, text_lengths = batch.review

          predictions = MLP_model(text).squeeze(1)

          loss = loss_func(predictions, batch.sentiment)

          acc = accuracy(predictions, batch.sentiment)

          valid_epoch_loss += loss.item()
          valid_epoch_acc += acc.item()

   

  if valid_epoch_loss < best_valid_loss:
            best_valid_loss = valid_epoch_loss
            torch.save(MLP_model.state_dict(), 'saved_weights'+'_linear.pt')

  print(f'\tTrain Loss: {train_epoch_loss / len(train_iterator):.3f} | Train Acc: {train_epoch_acc  / len(train_iterator)* 100:.2f}%')
  print(f'\t Val. Loss: {valid_epoch_loss / len(valid_iterator):.3f} |  Val. Acc: {valid_epoch_acc / len(valid_iterator) * 100:.2f}%')

	Train Loss: 0.736 | Train Acc: 50.45%
	 Val. Loss: 0.692 |  Val. Acc: 50.98%
	Train Loss: 0.649 | Train Acc: 57.34%
	 Val. Loss: 0.689 |  Val. Acc: 53.50%
	Train Loss: 0.547 | Train Acc: 63.12%
	 Val. Loss: 0.801 |  Val. Acc: 52.84%
	Train Loss: 0.526 | Train Acc: 62.24%
	 Val. Loss: 0.952 |  Val. Acc: 54.25%
	Train Loss: 0.515 | Train Acc: 63.58%
	 Val. Loss: 0.855 |  Val. Acc: 52.77%
	Train Loss: 0.517 | Train Acc: 63.37%
	 Val. Loss: 0.941 |  Val. Acc: 53.21%
	Train Loss: 0.511 | Train Acc: 64.07%
	 Val. Loss: 1.019 |  Val. Acc: 53.14%
	Train Loss: 0.524 | Train Acc: 63.23%
	 Val. Loss: 1.028 |  Val. Acc: 52.94%
	Train Loss: 0.518 | Train Acc: 63.88%
	 Val. Loss: 0.790 |  Val. Acc: 53.83%
	Train Loss: 0.504 | Train Acc: 64.44%
	 Val. Loss: 1.023 |  Val. Acc: 53.89%
	Train Loss: 0.488 | Train Acc: 67.57%
	 Val. Loss: 0.994 |  Val. Acc: 54.51%
	Train Loss: 0.452 | Train Acc: 72.59%
	 Val. Loss: 0.854 |  Val. Acc: 70.75%
	Train Loss: 0.284 | Train Acc: 86.98%
	 Val. Loss: 0.788 |  Val

In [41]:
best_valid_loss = float('inf')
optimizer = torch.optim.Adam(LinearRmodel.parameters(), lr=lr)

loss_func = nn.CrossEntropyLoss()

for epoch in range(LRnum_epochs):
  train_epoch_loss = 0
  train_epoch_acc = 0
  for batch in train_iterator:
      optimizer.zero_grad()
      # retrieve text and no. of words
      text, text_lengths = batch.review

      #feedforward
      # model.to(device)
      predictions = LinearRmodel(text).squeeze(1)
      
      
      loss = loss_func(predictions, batch.sentiment)

      acc = accuracy(predictions, batch.sentiment)

      # perform backpropagation
      loss.backward()

      optimizer.step()

      train_epoch_loss += loss.item()
      train_epoch_acc += acc.item()

  

  valid_epoch_loss = 0
  valid_epoch_acc = 0

  LinearRmodel.eval()

  with torch.no_grad():
      for batch in valid_iterator:
          text, text_lengths = batch.review

          predictions = LinearRmodel(text).squeeze(1)

          loss = loss_func(predictions, batch.sentiment)

          acc = accuracy(predictions, batch.sentiment)

          valid_epoch_loss += loss.item()
          valid_epoch_acc += acc.item()

   

  if valid_epoch_loss < best_valid_loss:
            best_valid_loss = valid_epoch_loss
            torch.save(LinearRmodel.state_dict(), 'saved_weights'+'_linear.pt')

  print(f'\tTrain Loss: {train_epoch_loss / len(train_iterator):.3f} | Train Acc: {train_epoch_acc  / len(train_iterator)* 100:.2f}%')
  print(f'\t Val. Loss: {valid_epoch_loss / len(valid_iterator):.3f} |  Val. Acc: {valid_epoch_acc / len(valid_iterator) * 100:.2f}%')

	Train Loss: 0.696 | Train Acc: 50.17%
	 Val. Loss: 0.693 |  Val. Acc: 50.29%
	Train Loss: 0.693 | Train Acc: 50.10%
	 Val. Loss: 0.693 |  Val. Acc: 49.71%
	Train Loss: 0.693 | Train Acc: 49.65%
	 Val. Loss: 0.693 |  Val. Acc: 50.29%
	Train Loss: 0.693 | Train Acc: 49.84%
	 Val. Loss: 0.693 |  Val. Acc: 50.29%
	Train Loss: 0.693 | Train Acc: 49.67%
	 Val. Loss: 0.693 |  Val. Acc: 50.29%
	Train Loss: 0.693 | Train Acc: 49.92%
	 Val. Loss: 0.693 |  Val. Acc: 50.29%
	Train Loss: 0.693 | Train Acc: 50.35%
	 Val. Loss: 0.693 |  Val. Acc: 50.29%
	Train Loss: 0.693 | Train Acc: 50.12%
	 Val. Loss: 0.693 |  Val. Acc: 50.29%
	Train Loss: 0.693 | Train Acc: 50.25%
	 Val. Loss: 0.693 |  Val. Acc: 50.29%
	Train Loss: 0.693 | Train Acc: 49.97%
	 Val. Loss: 0.693 |  Val. Acc: 49.71%
	Train Loss: 0.693 | Train Acc: 49.82%
	 Val. Loss: 0.693 |  Val. Acc: 50.29%
	Train Loss: 0.693 | Train Acc: 49.68%
	 Val. Loss: 0.693 |  Val. Acc: 50.29%
	Train Loss: 0.693 | Train Acc: 50.27%
	 Val. Loss: 0.693 |  Val